# Swiss Housing Price Predictor 🏠💰

Hello, we are Tobias, Sascha, and Florian. Our goal for this project is to build a predictive model for Swiss housing prices. We aim to achieve this by leveraging current web data. 

![Swiss House](https://archivaldesigns.com/cdn/shop/products/Peach-Tree-Front_1200x.jpg?v=1648224612)

This project will involve several steps:

1. **Data Collection**: Scrape housing data from various online sources.
2. **Preprocessing**: Clean and format the data for analysis.
3. **Exploratory Data Analysis**: Understand the data and find any interesting trends.
4. **Feature Engineering**: Create new features from the existing data to improve our model's performance.
5. **Model Building**: Train a machine learning model on our processed data.
6. **Evaluation**: Test our model's performance and make any necessary adjustments.

Our aim is that our model will be able to accurately predict housing prices, providing valuable insights for both buyers and sellers in the Swiss housing market. 

You can check out our code [here on GitHub](https://github.com/meichflo/ADS_Project_group3).

> "The only way to do great work is to love what you do." - Steve Jobs

### Imports

In [1]:
# Libraries
import os
import re
import json
import ast
import time
import random
import pandas as pd

from bs4 import BeautifulSoup

from prettytable import from_csv

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(f'Current working directory: {os.getcwd()}')

Current working directory: c:\Users\FMeic\data_science\ADS_Project_group3


# Data Collection
In this phase, we focus on gathering the necessary data for our project. We start by identifying the data sources, which can include databases, APIs, and public datasets. Once we have determined where the data will come from, we proceed with the data acquisition process. This involves collecting the data through various methods such as web scraping, querying databases, or accessing data through APIs.

### Web Scraping from Immoscout24
As part of our data collection efforts for the immo-project, we will be utilizing web scraping techniques to extract relevant information from the swiss immo plattform **Immoscout24**. This will allow us to gather valuable data that will contribute to our project.

In [ ]:
# List of user agents to rotate through
ua_list = ["Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
           "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
           "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:88.0) Gecko/20100101 Firefox/88.0"]

# Set up the Chrome driver with a random user agent and headless mode
opts = Options()
opts.add_argument("--headless")
opts.add_argument("user-agent=" + random.choice(ua_list))

#service = Service(executable_path=r'C:\Tools\chromedriver\chromedriver.exe')
service = Service(executable_path=r'/Users/tobiaskoller/Library/CloudStorage/OneDrive-ZHAW/2. Semester/Applied Data Science/tools/chromedriver-mac-x64/chromedriver')

# Chrome-Driver
driver = webdriver.Chrome(service=service, options=opts)

# Base URL of the website to scrape
base_url = 'https://www.immoscout24.ch/de/immobilien/mieten/kanton-zuerich?pn='

# Initialize an empty list to store scraped data
all_detail_urls = []


current_page = 1    # Start with page 1 of the search results
max_pages = 50  # Specify the maximum number of pages to scrape

# Loop through all pages to extract URLs of property listings
while current_page <= max_pages:
    driver.get(base_url + str(current_page))     # Load the website
    source = driver.page_source                  # Get the full content of the website
    soup = BeautifulSoup(source, 'html.parser')  # Parse HTML content with BeautifulSoup
    
    # Extract detail page URLs
    listing_links_elements = soup.find_all('a', class_='HgCardElevated_content_uir_2 HgCardElevated_link_EHfr7')
    detail_urls = [link['href'] for link in listing_links_elements if 'href' in link.attrs]
    # Assuming the href might not be a full URL, prepend base URL if needed
    full_urls = ['https://www.immoscout24.ch' + url if not url.startswith('http') else url for url in detail_urls]
    all_detail_urls.extend(full_urls)

    # Increment the page number
    current_page += 1

# Initialize empty lists to store the scraped data
all_detailed_id = []
all_detailed_title = []
all_detailed_address = []
all_detailed_price = []
all_detailed_rooms = []
all_detailed_squareMeters = []
all_detailed_descriptions = [] 
all_detailed_coreAttributes = []
all_detailed_features = []


test_urls = ['https://www.immoscout24.ch/mieten/4000855763', 'https://www.immoscout24.ch/mieten/4000894477']

# Loop through all detail URLs to extract detailed information
for url in all_detail_urls:
    driver.get(url)
    detailed_source = driver.page_source
    detailed_soup = BeautifulSoup(detailed_source, 'html.parser')

    detailed_id = re.search(r'\d+', url).group()

    detailed_title_element = detailed_soup.find('h1', class_='ListingTitle_spotlightTitle_ENVSi')
    detailed_title = detailed_title_element.text.strip() if detailed_title_element else 'No title found'
    all_detailed_title.append(detailed_title)

    detailed_address_element = detailed_soup.find('address', class_='AddressDetails_address_i3koO')
    detailed_address = detailed_address_element.text.strip() if detailed_address_element else 'No address found'
    all_detailed_address.append(detailed_address)

    detailed_price_element = detailed_soup.find('div', class_='SpotlightAttributesPrice_value_TqKGz')
    detailed_price = detailed_price_element.text.strip() if detailed_price_element else 'No price found'
    all_detailed_price.append(detailed_price)

    detailed_rooms_element = detailed_soup.find('div', class_='SpotlightAttributesNumberOfRooms_value_TUMrd')
    detailed_rooms = detailed_rooms_element.text.strip() if detailed_rooms_element else 'No rooms found'
    all_detailed_rooms.append(detailed_rooms)

    detailed_squareMeters_element = detailed_soup.find('div', class_='SpotlightAttributesUsableSpace_value_cpfrh')
    detailed_squareMeters = detailed_squareMeters_element.text.strip() if detailed_squareMeters_element else 'No square meters found'
    all_detailed_squareMeters.append(detailed_squareMeters)

    detailed_description_element = detailed_soup.find('div', class_='Description_descriptionBody_AYyuy')
    detailed_description = detailed_description_element.text.strip() if detailed_description_element else 'No description found'
    all_detailed_descriptions.append(detailed_description)

    detailed_coreAttributes_element = detailed_soup.find('div', class_='CoreAttributes_coreAttributes_e2NAm')
    #detailed_coreAttributes = detailed_coreAttributes_element.text.strip() if detailed_coreAttributes_element else 'No Core Attributes found'
    #all_detailed_coreAttributes.append(detailed_coreAttributes)

    attributes = {}
    if detailed_coreAttributes_element is None:
        all_detailed_coreAttributes.append({'wartning': 'No core attributes found'})
    else:
        for dt, dd in zip(detailed_coreAttributes_element.find_all('dt'), detailed_coreAttributes_element.find_all('dd')):
            key = dt.text.strip(':')
            value = ' '.join(dd.text.split())  # This is to clean up the value, removing extra spaces and newlines
            attributes[key] = value
        all_detailed_coreAttributes.append(attributes)


    features_list = detailed_soup.find('ul', class_='FeaturesFurnishings_list_S54KV')
    features = []
    if features_list is None:
        features.append('No features found')
    else:
        for feature_item in features_list.find_all('li'):
            # Extract the text from the p tag within each li
            feature_text = feature_item.find('p').text.strip()
            features.append(feature_text)
    all_detailed_features.append(features)

    time.sleep(random.uniform(2, 4))

# Close the driver after scraping is done
driver.close()

# Convert the scraped data to a DataFrame
df = pd.DataFrame({
    'Title': all_detailed_title,
    'Address': all_detailed_address,
    'Price': all_detailed_price,
    'Rooms': all_detailed_rooms,
    'SquareMeters': all_detailed_squareMeters,
    'CoreAttributes': all_detailed_coreAttributes,
    'Features': all_detailed_features,
    'Description': all_detailed_descriptions,
    'URL': all_detail_urls
                    })

# Get the current date
current_date = datetime.now().date()

# Add the current date to each entry in the DataFrame
df['Date'] = current_date

csvName = 'immoscout24_zh.csv'

# Save to file
df.to_csv(csvName, sep=";", index=False)

# Show file
with open(csvName) as table_file:
    tab = from_csv(table_file, delimiter=';')
    
    # Set the alignment for all columns to 'l' (left-align)
    for field_name in tab.field_names:
        tab.align[field_name] = 'l'
    
    print(tab)

### Cleansing of Scraped Data

In [8]:
df_cleansing = pd.read_csv('immoscout24_zh_all_20240402.csv', sep=';')
all_coreAttributes_list_strings = df_cleansing['CoreAttributes'].tolist()
all_features_list_strings = df_cleansing['Features'].tolist()
all_features_list = [json.loads(f_strings.replace("'", '"')) for f_strings in all_features_list_strings]


# Price Data Cleansing
def convert_price_to_number(price_str):
    cleaned_price = ''.join([char for char in price_str if char.isdigit()]) # Remove all characters except digits 
    if cleaned_price:
        return int(cleaned_price) # Convert the cleaned string to an integer
    else:
        return None  # Return None or 0 if the price is not found or cannot be converted  
# Apply the function to the 'Price' column
df_cleansing['Price'] = df_cleansing['Price'].apply(convert_price_to_number)


# Square Meters Data Cleansing
def convert_square_meters_to_number(square_meters_str):
    cleaned_square_meters = square_meters_str.replace('m2', '').strip() # Remove 'm2' and any surrounding whitespace
    if cleaned_square_meters.isdigit():
        return int(cleaned_square_meters) # Convert the cleaned string to an integer
    else:
        return None  # Return None or 0 if the square meter is not found or cannot be converted
# Apply the function to the 'SquareMeters' column
df_cleansing['SquareMeters'] = df_cleansing['SquareMeters'].apply(convert_square_meters_to_number)
    
    
# Split Core Attributes Data
def convert_strings_to_dicts(cA_strings):
    converted_cA_dicts = []
    for d in cA_strings:
        try:
            # Attempt to convert string to dictionary
            converted_cA_dict = ast.literal_eval(d)
            converted_cA_dicts.append(converted_cA_dict)
        except Exception as e:
            # Print problematic string and error message
            print("Error converting string:", d)
            print("Error message:", str(e))
    return converted_cA_dicts

# Apply function to convert strings to dictionaries
all_detailed_coreAttributes_list = convert_strings_to_dicts(all_coreAttributes_list_strings)

df_updates = pd.DataFrame(all_detailed_coreAttributes_list)
df_cleansing = pd.concat([df_cleansing, df_updates], axis=1)


# Split Features Data
def get_unique_values(all_detailed_features_input): # Function to get all unique values in the features list
    unique_fatures = set()
    for sublist in all_detailed_features_input:
        for features in sublist:
            unique_fatures.update(sublist)
    return list(unique_fatures)

# Apply function to get all unique values in the 'Features' column
unique_attributes = get_unique_values(all_features_list)

feature_rows_list = []
for array in all_features_list:
    feature_row = {attribute: False for attribute in unique_attributes} # Initialize a row with all empty strings
    # Mark with 'X' all attributes present in the current array
    for attribute in array:
        if attribute in feature_row:
            feature_row[attribute] = True
    # Append the row to the list
    feature_rows_list.append(feature_row)

# Create a DataFrame from the rows_list and then concatenate it with the original df
df_cleansing = pd.concat([df_cleansing, pd.DataFrame(feature_rows_list)], axis=1)

# Drop the original 'CoreAttributes' and 'Features' columns
df_cleansing.drop(['CoreAttributes', 'Features'], axis=1, inplace=True)

# Save the cleansed DataFrame to a new CSV file
df_cleansing.to_csv('immoscout24_zh_cleansed.csv', index=False, sep=';')
print('Data cleansing completed and saved to immoscout24_zh_all_20240402_cleansed.csv')


Error converting string: No core attributes found
Error message: invalid syntax (<unknown>, line 1)
Error converting string: No core attributes found
Error message: invalid syntax (<unknown>, line 1)
Error converting string: No core attributes found
Error message: invalid syntax (<unknown>, line 1)
Data cleansing completed and saved to immoscout24_zh_all_20240402_cleansed.csv


In [10]:
df_cleansing.head()

,Title,Address,Price,Rooms,SquareMeters,Description,URL,Date,Verfügbarkeit,Objekttyp,...,Aussicht,Reduit,No features found,Abwasseranschluss,Swimmingpool,Eckhaus,Kinderfreundlich,Wasseranschluss,Geschirrspüler,In Wohngemeinschaft
0,"""Moderne Wohnung mit einzigartigem Panorama""","Bahnhofweg 1, 8302 Kloten",2570.0,2.5,73.0,"Innovation, Komfort und Weitsicht - Wohnen in ...",https://www.immoscout24.ch/mieten/4000939090,2024-04-02,16.05.2024,Wohnung,...,True,False,False,False,False,False,False,False,False,False
1,"""ACHTUNG: Wohnperle sucht Mieter!""","Herrligweg 9, 8048 Zürich",1921.0,1.5,26.0,"Wohnperle an ruhiger, zentraler Lage sucht Mie...",https://www.immoscout24.ch/mieten/4000598194,2024-04-02,Sofort,Wohnung,...,False,False,False,False,False,False,False,False,False,False
2,"""ERSTVERMIETUNG - ruhig, grün, urban - erstkla...","Poststrasse 47, 8953 Dietikon",2830.0,3.5,104.0,Per Herbst 2024 entstehen insgesamt 39 erstkla...,https://www.immoscout24.ch/mieten/4000895944,2024-04-02,01.10.2024,Wohnung,...,False,False,False,False,False,False,True,False,False,False
3,"""Hochwertig ausgebaute 3.5-Zimmerwohnung - B-1.1""","Im Steinacher 3, 8303 Bassersdorf",2950.0,3.5,90.0,Für Anspruchsvolle: hochwertig ausgebaute 3.5-...,https://www.immoscout24.ch/mieten/4000679565,2024-04-02,Nach Vereinbarung,Wohnung,...,False,False,False,False,False,False,True,False,False,False
4,"""Erstvermietung Neubau Wohnungen an zentraler ...","Widenstrasse 1, 8302 Kloten",2950.0,3.5,92.0,Wir vermieten per 1. Juli 2024 an ruhiger Lage...,https://www.immoscout24.ch/mieten/4000823521,2024-04-02,01.07.2024,Wohnung,...,True,False,False,False,False,False,True,False,False,False


# Data Cleaning and Preprocessing

### Data Reduction
If necessary, we will reduce the dimensionality or data volume to improve performance. This step can help to simplify our data and make our subsequent analyses more efficient.

We get rid of the following feautures, since they are not valuable for a ML model:
- Title                    
- Address                  
- Description              
- URL                      
- Date  
- Verfügbarkeit
- Anzahl Etagen

In [ ]:
df_cleansing = pd.read_csv('immoscout24_zh_cleansed.csv', sep=';')

### Data Transformation
In the following cells we normalize our data, handle categorical data, and create new variables if necessary. This step is essential for preparing our data for machine learning algorithms, which often require data to be in a specific format.

In [3]:
df_cleansing['Rooms'] = pd.to_numeric(df_cleansing['Rooms'], errors='coerce')
df_cleansing['Date'] = pd.to_datetime(df_cleansing['Date'])

In [4]:
df_cleansing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Title                    1000 non-null   object        
 1   Address                  1000 non-null   object        
 2   Price                    985 non-null    float64       
 3   Rooms                    872 non-null    float64       
 4   SquareMeters             870 non-null    float64       
 5   Description              996 non-null    object        
 6   URL                      1000 non-null   object        
 7   Date                     1000 non-null   datetime64[ns]
 8   Verfügbarkeit            997 non-null    object        
 9   Objekttyp                997 non-null    object        
 10  Anzahl Zimmer            872 non-null    float64       
 11  Etage                    751 non-null    object        
 12  Anzahl Etagen            134 non-nu

get rid of unusable Features:
- Title                    
- Address                  
- Description              
- URL                      
- Date  
- Verfügbarkeit
- Anzahl Etagen

Must be Categorical:
- Objekttyp   
    ['Wohnung', 'Maisonette / Duplex', 'Loft', 'Attikawohnung',
       'Reihenfamilienhaus', 'Einfamilienhaus', 'Dachwohnung',
       'Hobbyraum', 'Einzelzimmer', 'Terrassenwohnung', 'Studio',
       'Doppeleinfamilienhaus', 'Bauernhaus', 'Schloss',
       'Mehrfamilienhaus', 'Villa', 'Kellerabteil', 'Terrassenhaus', nan]       

must be changed:
- Etage 
    - eg = 0   
- remove unit ("m2", "m"
    - Wohnfläche 
    - Nutzfläche
    - Raumhöhe
    - Grundstückfläche
    - Kubatur
    - Hallenhöhe
    - Mindestnutzfläche
- remove coma (transform to int)
    - Baujahr 
    - Anzahl Wohnungen
    - Letztes Renovationsjahr
    - Anzahl Toiletten

In [22]:
df_cleansing['Mindestnutzfläche'].unique()

array([nan, '18 m2'], dtype=object)

In [12]:
nan_count = df_cleansing['Address'].isna().sum()
print(nan_count)

0


# Feature Engineering


### Get coordinates for adresses

To get the coordinates for the addresses we use the official API from the Federal Office of Topography swisstopo: https://api3.geo.admin.ch

In [13]:
# add the coordinates to the dataframe
from coordinates import get_coordinates
df_cleansing['Coordinates'] = df_cleansing['Address'].apply(get_coordinates)

In [14]:
df_cleansing[['Latitude', 'Longitude']] = df_cleansing['Coordinates'].apply(pd.Series)

In [15]:
df_cleansing.head()

,Title,Address,Price,Rooms,SquareMeters,Description,URL,Date,Verfügbarkeit,Objekttyp,...,Abwasseranschluss,Swimmingpool,Eckhaus,Kinderfreundlich,Wasseranschluss,Geschirrspüler,In Wohngemeinschaft,Coordinates,Latitude,Longitude
0,"""Moderne Wohnung mit einzigartigem Panorama""","Bahnhofweg 1, 8302 Kloten",2570.0,2.5,73.0,"Innovation, Komfort und Weitsicht - Wohnen in ...",https://www.immoscout24.ch/mieten/4000939090,2024-04-02,16.05.2024,Wohnung,...,False,False,False,False,False,False,False,"(47.4499626159668, 8.582908630371094)",47.449963,8.582909
1,"""ACHTUNG: Wohnperle sucht Mieter!""","Herrligweg 9, 8048 Zürich",1921.0,1.5,26.0,"Wohnperle an ruhiger, zentraler Lage sucht Mie...",https://www.immoscout24.ch/mieten/4000598194,2024-04-02,Sofort,Wohnung,...,False,False,False,False,False,False,False,"(47.389892578125, 8.484477996826172)",47.389893,8.484478
2,"""ERSTVERMIETUNG - ruhig, grün, urban - erstkla...","Poststrasse 47, 8953 Dietikon",2830.0,3.5,104.0,Per Herbst 2024 entstehen insgesamt 39 erstkla...,https://www.immoscout24.ch/mieten/4000895944,2024-04-02,01.10.2024,Wohnung,...,False,False,False,True,False,False,False,"(47.3998908996582, 8.403274536132812)",47.399891,8.403275
3,"""Hochwertig ausgebaute 3.5-Zimmerwohnung - B-1.1""","Im Steinacher 3, 8303 Bassersdorf",2950.0,3.5,90.0,Für Anspruchsvolle: hochwertig ausgebaute 3.5-...,https://www.immoscout24.ch/mieten/4000679565,2024-04-02,Nach Vereinbarung,Wohnung,...,False,False,False,True,False,False,False,"(47.43076705932617, 8.642346382141113)",47.430767,8.642346
4,"""Erstvermietung Neubau Wohnungen an zentraler ...","Widenstrasse 1, 8302 Kloten",2950.0,3.5,92.0,Wir vermieten per 1. Juli 2024 an ruhiger Lage...,https://www.immoscout24.ch/mieten/4000823521,2024-04-02,01.07.2024,Wohnung,...,False,False,False,True,False,False,False,"(47.446773529052734, 8.579062461853027)",47.446774,8.579062


#### Check the Data quality:
As the addresses in this dataset were collected through web scraping, it is important to consider the potential for inaccuracy in the data. To assess the completeness of the data, we must first determine the extent to which user input may have influenced the final result. 

In [16]:
print("How many % without coordinates?\n",df_cleansing['Coordinates'].value_counts()[(None, None)]*100/df_cleansing.shape[0], "%")

How many % without coordinates?
 12.0 %


12% of the addresses could not be converted into coordinates. 

---
Let's save the data to .csv so my friends don't have to run the cell with the API requests. 

In [17]:
df_cleansing.to_csv('immoscout24_zh_with_coordinates.csv', index=False, sep=';')

In [18]:
df_w_coordinates = pd.read_csv('immoscout24_zh_with_coordinates.csv', sep=';')
df_w_coordinates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 62 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Title                    1000 non-null   object 
 1   Address                  1000 non-null   object 
 2   Price                    985 non-null    float64
 3   Rooms                    872 non-null    float64
 4   SquareMeters             870 non-null    float64
 5   Description              996 non-null    object 
 6   URL                      1000 non-null   object 
 7   Date                     1000 non-null   object 
 8   Verfügbarkeit            997 non-null    object 
 9   Objekttyp                997 non-null    object 
 10  Anzahl Zimmer            872 non-null    float64
 11  Etage                    751 non-null    object 
 12  Anzahl Etagen            134 non-null    float64
 13  Wohnfläche               873 non-null    object 
 14  Baujahr                  

### Compute distance to next trainstation
Bellow we add the comuted feature **Distance to station** to the dataframe. 

Since the earth is not flat, the Pythagorean theorem would not be accurate for calculating distances between geographical coordinates. The Haversine formula or other spherical geometry methods are more appropriate for that purpose.


![Image description](resources\Haversine.png)

[Image Source](https://www.google.com/imgres?imgurl=https%3A%2F%2Fwww.vcalc.com%2Fattachments%2Fe6d11679-da27-11e2-8e97-bc764e04d25f%2F220px-Law-of-haversines.svg.png&tbnid=IWPxBR6bJWrqlM&vet=12ahUKEwjcuMO1-JmFAxU9kP0HHZ3sBuoQMygQegQIARBu..i&imgrefurl=https%3A%2F%2Fwww.vcalc.com%2Fwiki%2Fvcalc%2Fhaversine-distance&docid=MOeiC0D7gb9SqM&w=220&h=220&q=Haversine%20formula&ved=2ahUKEwjcuMO1-JmFAxU9kP0HHZ3sBuoQMygQegQIARBu)

In [19]:
from distance_to_station import get_distance_to_station
df_w_coordinates['DistanceToStationKM'] = df_w_coordinates.apply(lambda row: get_distance_to_station(row['Latitude'], row['Longitude']), axis=1)

In [20]:
df_w_coordinates.head()

,Title,Address,Price,Rooms,SquareMeters,Description,URL,Date,Verfügbarkeit,Objekttyp,...,Swimmingpool,Eckhaus,Kinderfreundlich,Wasseranschluss,Geschirrspüler,In Wohngemeinschaft,Coordinates,Latitude,Longitude,DistanceToStationKM
0,"""Moderne Wohnung mit einzigartigem Panorama""","Bahnhofweg 1, 8302 Kloten",2570.0,2.5,73.0,"Innovation, Komfort und Weitsicht - Wohnen in ...",https://www.immoscout24.ch/mieten/4000939090,2024-04-02,16.05.2024,Wohnung,...,False,False,False,False,False,False,"(47.4499626159668, 8.582908630371094)",47.449963,8.582909,0.447263
1,"""ACHTUNG: Wohnperle sucht Mieter!""","Herrligweg 9, 8048 Zürich",1921.0,1.5,26.0,"Wohnperle an ruhiger, zentraler Lage sucht Mie...",https://www.immoscout24.ch/mieten/4000598194,2024-04-02,Sofort,Wohnung,...,False,False,False,False,False,False,"(47.389892578125, 8.484477996826172)",47.389893,8.484478,4.425504
2,"""ERSTVERMIETUNG - ruhig, grün, urban - erstkla...","Poststrasse 47, 8953 Dietikon",2830.0,3.5,104.0,Per Herbst 2024 entstehen insgesamt 39 erstkla...,https://www.immoscout24.ch/mieten/4000895944,2024-04-02,01.10.2024,Wohnung,...,False,False,True,False,False,False,"(47.3998908996582, 8.403274536132812)",47.399891,8.403275,0.667844
3,"""Hochwertig ausgebaute 3.5-Zimmerwohnung - B-1.1""","Im Steinacher 3, 8303 Bassersdorf",2950.0,3.5,90.0,Für Anspruchsvolle: hochwertig ausgebaute 3.5-...,https://www.immoscout24.ch/mieten/4000679565,2024-04-02,Nach Vereinbarung,Wohnung,...,False,False,True,False,False,False,"(47.43076705932617, 8.642346382141113)",47.430767,8.642346,1.491088
4,"""Erstvermietung Neubau Wohnungen an zentraler ...","Widenstrasse 1, 8302 Kloten",2950.0,3.5,92.0,Wir vermieten per 1. Juli 2024 an ruhiger Lage...,https://www.immoscout24.ch/mieten/4000823521,2024-04-02,01.07.2024,Wohnung,...,False,False,True,False,False,False,"(47.446773529052734, 8.579062461853027)",47.446774,8.579062,0.757422


In [21]:
df_w_coordinates.to_csv('immoscout24_zh_with_coordinates&distance.csv', index=False, sep=';')

In [22]:
df_w_coordinates_distance = pd.read_csv('immoscout24_zh_with_coordinates&distance.csv', sep=';')
df_w_coordinates_distance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 63 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Title                    1000 non-null   object 
 1   Address                  1000 non-null   object 
 2   Price                    985 non-null    float64
 3   Rooms                    872 non-null    float64
 4   SquareMeters             870 non-null    float64
 5   Description              996 non-null    object 
 6   URL                      1000 non-null   object 
 7   Date                     1000 non-null   object 
 8   Verfügbarkeit            997 non-null    object 
 9   Objekttyp                997 non-null    object 
 10  Anzahl Zimmer            872 non-null    float64
 11  Etage                    751 non-null    object 
 12  Anzahl Etagen            134 non-null    float64
 13  Wohnfläche               873 non-null    object 
 14  Baujahr                  

### Add feature sun hours
It might be interesting to have a feature in the dataframe, which represents how sunny it is on the location in question. 
Therefore we add the average sun hours per day of the location to the dataframe. 

In [23]:
from weather import get_sunshine_duration
df_w_coordinates_distance['Weather'] = df_w_coordinates_distance.apply(lambda row: get_sunshine_duration(row['Latitude'], row['Longitude']), axis=1)


Getting sunshine duration data for 47.4499626159668 8.582908630371094
Getting sunshine duration data for 47.389892578125 8.484477996826172
Getting sunshine duration data for 47.3998908996582 8.403274536132812
Getting sunshine duration data for 47.43076705932617 8.642346382141113
Getting sunshine duration data for 47.446773529052734 8.579062461853027
Getting sunshine duration data for 47.20563888549805 8.710277557373047
Getting sunshine duration data for 47.400333404541016 8.403396606445312
Getting sunshine duration data for 47.31671905517578 8.792036056518555
Getting sunshine duration data for 47.38902282714844 8.492106437683105
Getting sunshine duration data for 47.4499626159668 8.582908630371094
Getting sunshine duration data for 47.36667251586914 8.828701972961426
Getting sunshine duration data for 47.399173736572266 8.600276947021484
Getting sunshine duration data for nan nan
Getting sunshine duration data for 47.396705627441406 8.528757095336914
Getting sunshine duration data for 

In [24]:
df_w_coordinates_distance.head(13) # Show the first 13 rows - keep an eye on the 13th row

,Title,Address,Price,Rooms,SquareMeters,Description,URL,Date,Verfügbarkeit,Objekttyp,...,Eckhaus,Kinderfreundlich,Wasseranschluss,Geschirrspüler,In Wohngemeinschaft,Coordinates,Latitude,Longitude,DistanceToStationKM,Weather
0,"""Moderne Wohnung mit einzigartigem Panorama""","Bahnhofweg 1, 8302 Kloten",2570.0,2.5,73.0,"Innovation, Komfort und Weitsicht - Wohnen in ...",https://www.immoscout24.ch/mieten/4000939090,2024-04-02,16.05.2024,Wohnung,...,False,False,False,False,False,"(47.4499626159668, 8.582908630371094)",47.449963,8.582909,0.447263,28213.432995
1,"""ACHTUNG: Wohnperle sucht Mieter!""","Herrligweg 9, 8048 Zürich",1921.0,1.5,26.0,"Wohnperle an ruhiger, zentraler Lage sucht Mie...",https://www.immoscout24.ch/mieten/4000598194,2024-04-02,Sofort,Wohnung,...,False,False,False,False,False,"(47.389892578125, 8.484477996826172)",47.389893,8.484478,4.425504,28213.432995
2,"""ERSTVERMIETUNG - ruhig, grün, urban - erstkla...","Poststrasse 47, 8953 Dietikon",2830.0,3.5,104.0,Per Herbst 2024 entstehen insgesamt 39 erstkla...,https://www.immoscout24.ch/mieten/4000895944,2024-04-02,01.10.2024,Wohnung,...,False,True,False,False,False,"(47.3998908996582, 8.403274536132812)",47.399891,8.403275,0.667844,27709.868874
3,"""Hochwertig ausgebaute 3.5-Zimmerwohnung - B-1.1""","Im Steinacher 3, 8303 Bassersdorf",2950.0,3.5,90.0,Für Anspruchsvolle: hochwertig ausgebaute 3.5-...,https://www.immoscout24.ch/mieten/4000679565,2024-04-02,Nach Vereinbarung,Wohnung,...,False,True,False,False,False,"(47.43076705932617, 8.642346382141113)",47.430767,8.642346,1.491088,28069.702060
4,"""Erstvermietung Neubau Wohnungen an zentraler ...","Widenstrasse 1, 8302 Kloten",2950.0,3.5,92.0,Wir vermieten per 1. Juli 2024 an ruhiger Lage...,https://www.immoscout24.ch/mieten/4000823521,2024-04-02,01.07.2024,Wohnung,...,False,True,False,False,False,"(47.446773529052734, 8.579062461853027)",47.446774,8.579062,0.757422,28213.432995
5,"""4.5 Zimmerwohnung in Direktorenvilla aus dem ...","Seestrasse 9, 8805 Richterswil",3900.0,4.5,160.0,Diese historische Backsteinvilla steht unter D...,https://www.immoscout24.ch/mieten/4000825604,2024-04-02,Nach Vereinbarung,Maisonette / Duplex,...,False,True,False,False,False,"(47.20563888549805, 8.710277557373047)",47.205639,8.710278,0.369995,28058.850440
6,"""ERSTVERMIETUNG - ruhig, grün, urban - erstkla...","Poststrasse 45, 8953 Dietikon",2130.0,2.5,66.0,Per Herbst 2024 entstehen insgesamt 39 erstkla...,https://www.immoscout24.ch/mieten/4000895943,2024-04-02,01.10.2024,Wohnung,...,False,True,False,False,False,"(47.400333404541016, 8.403396606445312)",47.400333,8.403397,0.617793,28213.432995
7,"""Exklusive, sehr grosse 2.5-Zimmer-Wohnung in ...","Guyer-Zeller-Strasse 17, 8620 Wetzikon",2495.0,2.5,78.0,"Wo einst die historische Villa gestanden hat, ...",https://www.immoscout24.ch/mieten/4000920773,2024-04-02,Sofort,Wohnung,...,False,False,False,False,False,"(47.31671905517578, 8.792036056518555)",47.316719,8.792036,0.147062,27182.299203
8,"""Einzigartige Loftwohnung im 12ten Himmel""","Albulastrasse 60, 8048 Zürich",4835.0,3.5,180.0,Ihre Wohlfühloase liegt uns am Herzen!Die zwei...,https://www.immoscout24.ch/mieten/4000897339,2024-04-02,Nach Vereinbarung,Loft,...,False,False,False,False,False,"(47.38902282714844, 8.492106437683105)",47.389023,8.492106,3.849733,28213.432995
9,"""LETZTE CHANCE auf Ihren Wohntraum mit Ausblick!""","Bahnhofweg 1, 8302 Kloten",2920.0,2.5,68.0,"Innovation, Komfort und Weitsicht - Wohnen in ...",https://www.immoscout24.ch/mieten/4000895605,2024-04-02,16.05.2024,Wohnung,...,False,False,False,False,False,"(47.4499626159668, 8.582908630371094)",47.449963,8.582909,0.447263,28213.432995


In [25]:
df_w_coordinates_distance.to_csv('immoscout24_zh_with_coordinates&distance&weather.csv', index=False, sep=';')

# Exploratory Data Analysis

We want to have a solid understanding of our dataset's characteristics which can help us to make informed decisions in the subsequent stages of our project. Therefore we do a **correlation analysis** to understand to correlataions within the data. Since we know the power of visual representation of data a **Visualization** is important. Use plots and charts to get a visual understanding of the data distributions and relationships is the goal for the follwing cells. To compare the data we first bring the features to the same scale.

- Feature Scaling (to have all the features on the same scale)
    - `from sklearn.preprocessing import StandardScaler`
- Do the correlation analysis
- filter attributes 
    - according to correlation analysis
- additionaly
    - geo map

##### Goal:
Find which parameters are useful and which can be ignored due to correlation. 

In [23]:
# correlation matrix

## Prepear the data

Before starting modeling we have to prepear the dataset accordingly. We have to split into train and test set. However for most of the models it is needed that label (y) and features (X) are separated as well. 


- Train/Test Split
    - 70/30
- Split X/y (Feature and Label)
    - `X_train = train_data.loc[:,train_data.columns != "Price"]`

In [24]:
# split the data

# Modeling
To address the regression problem of predicting the prices of houses we go through several steps while modeling.

# Model Evaluation

## Choose Model(s)
For our regression problem, we will evaluate and compare the following models to find the best fit for predicting house prices. The models include both linear and non-linear approaches to accommodate different patterns and complexities in the data.

### Models to Evaluate
1. **Random Forest Regressor**
   - A robust ensemble model that uses multiple decision trees to produce a more stable and accurate prediction. It's generally good at handling outliers and non-linear data.
   - Import with: `from sklearn.ensemble import RandomForestRegressor`

2. **Linear Regression**
   - A simple yet powerful model that assumes a linear relationship between input variables and the target. It's effective when the relationship is linear and is easy to interpret.
   - Import with: `from sklearn.linear_model import LinearRegression`

3. **Polynomial Regression**
   - An extension of linear regression that models the relationship as a polynomial. This allows the model to bend with data for non-linear trends while still remaining interpretable.
   - Polynomial Regression is essentially a special case of Linear Regression but involves creating polynomial features from the input variables.

4. **Neural Network (NN) Model using TensorFlow**
   - A highly flexible and powerful model capable of modeling complex non-linear relationships. Neural networks are particularly useful when we have a large amount of data and the relationship between variables is complex.
   - Implemented using TensorFlow, which provides the tools to build custom neural network architectures.

## Train Model
Each model will be trained using the same set of training data. This allows for a fair comparison between models based on their performance metrics.

- **Data Split**: Split the data into training and validation sets to evaluate the model's performance.
- **Training Process**: Fit each model to the training data using their respective training algorithms.

## Model Evaluation
Evaluate the performance of each model using the following criteria to ensure robustness and reliability:

- **Validation Techniques**: Implement cross-validation techniques such as K-fold cross-validation to validate the consistency and robustness of each model's performance across different subsets of the data.
- **Performance Metrics**: Assess the model using regression-specific metrics including:
  - RMSE (Root Mean Square Error): Measures the average magnitude of the errors in a set of predictions, without considering their direction. It’s particularly useful to compare prediction errors of different models and datasets.
  - MAE (Mean Absolute Error): Provides a measure of errors between paired observations expressing the same phenomenon.
  - R² (Coefficient of Determination): Indicates the proportion of the variance in the dependent variable that is predictable from the independent variables.
- **Iterate**: Based on the performance metrics, refine and iterate on the model's parameters to enhance prediction accuracy and align the model more closely with business needs.


## Model Selection
The final model selection will be based on a combination of performance metrics and the model's ability to generalize to unseen data. We aim to choose a model that not only performs well but also provides the right balance of complexity and interpretability suited for our use case.



### Meteo

In [26]:
# the command with the % will install the libraries directly in the correct enviroment
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [27]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [28]:
import openmeteo_requests

import matplotlib.pyplot as plt
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": latitude,
	"longitude": longitude,
	"start_date": "2024-03-01",
	"end_date": "2024-03-17",
	"daily": ["temperature_2m_mean", "sunshine_duration", "rain_sum"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")



# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(1).ValuesAsNumpy()
daily_rain_sum = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["sunshine_duration"] = daily_sunshine_duration /3600
daily_data["rain_sum"] = daily_rain_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

# Plotting
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Date')
plt.xticks(rotation=45)
ax1.set_ylabel('Temperature (°C)', color=color)
ax1.plot(daily_dataframe['date'], daily_dataframe['temperature_2m_mean'], color=color, label='Temperature')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('Sunshine Duration (hours)', color=color)
ax2.plot(daily_dataframe['date'], daily_dataframe['sunshine_duration'], color=color, label='Sunshine Duration')
ax2.tick_params(axis='y', labelcolor=color)

ax3 = ax1.twinx()  # instantiate a third axes for the rain sum
ax3.spines['right'].set_position(('outward', 60))  # Offset the right spine of ax3
color = 'tab:green'
ax3.set_ylabel('Rain Sum (mm)', color= "blue")
ax3.bar(daily_dataframe['date'], daily_dataframe['rain_sum'], color="blue", label='Rain Sum', alpha=0.5, width=0.5)
ax3.tick_params(axis='y', labelcolor="blue")

# Otherwise the right y-label is slightly clipped
fig.tight_layout()
plt.title('Weather Data Analysis')
plt.show()


NameError: name 'latitude' is not defined

In [ ]:
#Because my lovely group doesn't recognise my amazing work, I've finally implemented a clickable and interactive graph with Plotly.

In [ ]:
#install the following environment via your terminal.

#pip install plotly

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Assuming daily_dataframe is already created as per your Matplotlib example

# Create a subplot with 2 y-axes on the left and 1 y-axis on the right
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add temperature plot on primary y-axis
fig.add_trace(
    go.Scatter(x=daily_dataframe['date'], y=daily_dataframe['temperature_2m_mean'],
               name='Temperature (°C)', mode='lines', line=dict(color='red')),
    secondary_y=False,
)

# Add sunshine duration plot on secondary y-axis (left)
fig.add_trace(
    go.Scatter(x=daily_dataframe['date'], y=daily_dataframe['sunshine_duration'],
               name='Sunshine Duration (hours)', mode='lines', line=dict(color='blue')),
    secondary_y=True,
)

# Since Plotly does not directly support a third y-axis, we use a bar chart to represent the rain sum
# This will overlay on the primary y-axis but can be visually distinguished
fig.add_trace(
    go.Bar(x=daily_dataframe['date'], y=daily_dataframe['rain_sum'],
           name='Rain Sum (mm)', marker_color='green', opacity=0.5),
    secondary_y=False,
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Temperature (°C)", secondary_y=False, color='red')
fig.update_yaxes(title_text="Sunshine Duration (hours)", secondary_y=True, color='blue')

# Set layout options
fig.update_layout(
    title_text="Weather Data Analysis",
    xaxis_tickangle=-45,
    legend=dict(orientation="h"),
)

# Show plot
fig.show()


In [ ]:
#I hope my team is happy now.